In [ ]:
from selenium import webdriver
from xlwt import Workbook
import pandas as pd
import threading
import time


In [ ]:
#Defining Variable

loc=r'C:\Users\SHADOW\Downloads\chromedriver_win32\chromedriver.exe'
login='//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[4]'
user='//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[2]/div/label/input'
passw='//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[3]/div/label/input'
pic='//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/div[5]/a/img'
not_now='/html/body/div[4]/div/div/div[3]/button[2]'
xfollower='//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a'
xfollowing='//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a'
scroll='/html/body/div[4]/div/div[2]'
close='/html/body/div[4]/div/div[1]/div/div[2]/button/svg'
URL='https://www.instagram.com/'

no_of_following=0
no_of_followers=0
no_of_post=0
follower=set()
following=set()

In [ ]:
# Function to get 

In [ ]:
def opening_tab(username,password):
    driver=webdriver.Chrome(loc)      #used to locate the chrome driver in your pc
    driver.get(URL)                 #to open the website 
    time.sleep(3)                 #wait till site is being load
    driver.find_element_by_xpath(user).click()        #clicking on the username textfield
    driver.find_element_by_xpath(user).send_keys(username)  #giving input to textfield as given by user 
    driver.find_element_by_xpath(passw).click()       #clicking on the password textfield
    driver.find_element_by_xpath(passw).send_keys(password)        #giving password
    driver.find_element_by_xpath(login).click()   #clicking on the login button

    time.sleep(3)    #wait till it is loaded
    driver.find_element_by_xpath(not_now).click()    
    driver.find_element_by_xpath(pic).click()
    return driver

#for scrolling the list of followers and following and saving tht list
def scrolling(driver,x,total,s):
    
    driver.find_element_by_xpath(x).click()
    time.sleep(3)
    eula = driver.find_element_by_xpath(scroll)
    for i in range(total//10):
        for title in driver.find_elements_by_css_selector('a.FPmhX '):
                s.add(title.text)        
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', eula)
        time.sleep(3) 

#for getting the follower list 
def followers_func(username,password):
    driver1=opening_tab(username,password)
    time.sleep(2)
    no_of_post=driver1.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[1]/span/span').text
    no_of_followers=int(driver1.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a/span').text)
    scrolling(driver1,xfollower,no_of_followers,follower)
    
#for getting the following list
def following_func(username,password):
    driver2=opening_tab(username,password)
    time.sleep(2)
    no_of_following=int(driver2.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a/span').text)
    scrolling(driver2, xfollowing, no_of_following, following)



In [ ]:
#Main Code
username=input('Enter the username :')        #getting the username from user
password=input('Enter the password :')        #getting the password from user
try:
    t1 = threading.Thread(target=followers_func, name='t1',args=(username,password,))        #creating thread to achieve parallelism 
    t2 = threading.Thread(target=following_func, name='t2',args=(username,password,))        #thread1--->get the follower list
                                                                                         #thread2--->get the following list
    t1.start()    #start execution of thread1
    t2.start()    #start exectuion of thread2

    t1.join()     #main program also run along with the thread so join is used to wait until thread is being executed
    t2.join()

    mutual=list(follower&following)    #intersection of 2 set and converting it into list
    fans=list(follower-following)      #difference of 2 set 
    not_follow_back=list(following-follower)    

    # Workbook is created 
    wb = Workbook() 
  
    # add_sheet is used to create sheet. 
    sheet1 = wb.add_sheet(username) 

    #writing data to excel file  
    sheet1.write(1, 0, 'Follower = {}'.format(len(follower))) 
    sheet1.write(2, 0, 'Following = {}'.format(len(following))) 
    sheet1.write(3, 0, 'Friends = {}'.format(len(mutual))) 
    sheet1.write(4, 0, 'Fans = {}'.format(len(fans))) 
    sheet1.write(5, 0, 'Not Follow Back = {}'.format(len(not_follow_back))) 

    sheet1.write(8, 1, 'Friends') 
    sheet1.write(8, 2, 'Following') 
    sheet1.write(8, 3, 'Follower') 
    sheet1.write(8, 4, 'Fans') 
    sheet1.write(8, 5, 'Not Follow Back')

    list_all=[mutual,following,follower,fans,not_follow_back]
    z=1
    for l in list_all:
        for j,i in enumerate(l):
            sheet1.write(j+9, z, i)
        z=z+1
    
    wb.save('{}.xls'.format(username))
    print('File is created successfully...')
except:
    print("Sorry can't able to load....\nPlease check your internet connection")
    

